# Example: Create a new KDA info table

The BD wrapper contains a new prior that uses literature distance results to inform the P_far prior that helps resolve the kinematic distance ambiguity (KDA). In this notebook we show how to create a KDA info table that can be added to the already existing tables (which can be found in the `KDA_info` directory).

The current KDA info tables mostly cover regions in the first Galactic quadrant. See Appendix A in Riener et al. (2020b) for a discussion about the KDA prior and currently available KDA info data sets.  

In the following we give the code we used to produce a KDA info table from Table 1 of [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) and Table 2 of [Rathborne et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJS..182..131R/abstract) for a sample of molecular clouds from the Galactic Ring Survey.

In [ ]:
import os
import numpy as np
from astropy.table import Table, Column, join
from astropy import units as u


dirpath_kda_info = 'results'  # change this to 'os.path.join('..', 'KDA_info') for your own new tables'
kda_table_name = 'Roman-Duval+09'

#  check if dirpath_kda_info already exists, if not create it
if not os.path.exists(dirpath_kda_info):
    os.makedirs(dirpath_kda_info)

First we load the original tables:

In [ ]:
#  Table 1 from Roman-Duval+ 2009

t1 = Table.read(
    'https://cdsarc.unistra.fr/ftp/J/ApJ/699/1153/table1.dat',
    readme='https://cdsarc.unistra.fr/ftp/J/ApJ/699/1153/ReadMe',
    format="ascii.cds")

#  Table 2 from Rathborne+ 2009

t2 = Table.read(
    'https://cdsarc.unistra.fr/ftp/J/ApJS/182/131/table2.dat',
    readme='https://cdsarc.unistra.fr/ftp/J/ApJS/182/131/ReadMe',
    format="ascii.cds")

Table 1 from [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) contains the final kinematic distances to the Galactic clouds but does not explicitly state whether the near or far distance was chosen. In the following, we use classes and functions contained in the module `BD_wrapper.kinematic_distance` to infer the chosen kinematic distance solution.

In [ ]:
from BD_wrapper.kinematic_distance import KinematicDistance, infer_kinematic_distances


#  initialize the KinematicDistance object
kd = KinematicDistance()
kd.bdc_version = '2.4'
kd.initialize()

#  the f_avg column indicates whether the associated velocity dispersion value was determined
#  or measured for the individual source (0) or an average value was chosen (1). For the GRS
#  table the dispersion along the velocity axis was determined for each cloud, so we set the
#  entire f_avg column to 1.
f_avg = np.zeros(len(t1))
t1.add_column(Column(data=f_avg.astype('int'), name='f_avg'))

#  the in_grs columns just indicates whether the source is within the PPV region covered by the GRS.
#  This column can be ignored for other data sets.
within_grs = np.ones(len(t1))
t1.add_column(Column(data=within_grs.astype('int'), name='in_grs'))

#  infer the kinematic distance solutions and the chosen KDA resolution
#  Roman-Duval et al. (2009) assumed a distance to the Galactic center of 8.5 kpc, se we set R_0=8.5
dist_n, dist_f, kda = infer_kinematic_distances(t1['GLON'], t1['Dist'], R_0=8.5)

#  add the three new columns to Table 1 from Roman-Duval et al. (2009)
t1.add_column(Column(data=dist_n, name='dist_kd_n', format="{0:.2f}"))
t1.add_column(Column(data=dist_f, name='dist_kd_f', format="{0:.2f}"))
t1.add_column(Column(data=kda, name='KDA'))

Now we combine Table 1 of [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) and Table 2 of [Rathborne et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJS..182..131R/abstract) as we need information from both catalogues. 

In [ ]:
table = join(t1, t2, keys=['GRSMC'], join_type='left')

Finally, we can go on to produce the KDA info table. There are few things to consider here. To keep the structure to the other KDA info tables similar we suggest renaming the columns containing the names of the sources `'ID'` and the column containing the distances `'Dist_lit'`.

The spatial and spectral extent of the sources should be given in terms of their FWHM extent.

In [ ]:
from BD_wrapper.KDA_tables import kda_info_table, kda_info_table_ini

table.rename_column('GRSMC', 'ID')
table.rename_column('Dist', 'Dist_lit')

#  a: Major axis size of ellipse in deg {erroneously called semi-major axis in Rathborne+ 2009}
#  b: Minor axis size of ellipse in deg {erroneously called semi-minor axis in Rathborne+ 2009}
#  DelV: FWHM extent in velocity [km/s]
#  pa: Position angle measured w.r.t. the positive GLON axis -> offset_pa = 0

#  in the following we specify the corresponding column name in our merged table

#  Galactic longitude value in [deg]
glon = 'GLONc'
#  Galactic latitude value in [deg]
glat = 'GLATc'
#  VLSR position of the source in [km/s]
vlsr = 'Vlsr_1'
#  semi-major axis of source, the unit can be specified with unit_a
a = 'a'
#  semi-minor axis of source, the unit can be specified with unit_b
b = 'b'
#  position angle of the source; if pa is measured east of Galactic north 
pa = 'pa'
#  spectral extent of the source, typically given as the full width at half maximum (FWHM)
dvlsr = 'DelV'
#  column specifying whether the chosen literature distance correponds to the near 
#  or far kinematic distance solution (given as 'N' or 'F')
kda = 'KDA'

#  the following factors ease the homogenization for different catalogues as entries
#  can be for example given in terms of the FWHM extent or standard deviation

fwhm_factor = 2.354820045
#  if a is given as major axis -> factor_a = 0.5
#  if a is given as semi-major axis -> factor_a = 1.0
factor_a = 0.5
#  if b is given as minor axis -> factor_b = 0.5
#  if a is given as semi-minor axis -> factor_b = 1.0
factor_b = 0.5
#  if dvlsr is given as the spectral FWHM extent -> factor_dvlsr = 1
#  if dvlsr is given as velocity dispersion -> factor_dvlsr = fwhm_factor
factor_dvlsr = 1
#  if position angle is measured w.r.t. the positive GLON axis -> offset_pa = 0
#  if position angle is measured east of north -> offset_pa = 90
offset_pa = 0

#  specify the units of a, b, and dvlsr 
unit_a = u.deg
unit_b = u.deg
unit_dvlsr = u.km/u.s

#  make the KDA info table
table = kda_info_table(
    table, 
    glon=glon, 
    glat=glat, 
    vlsr=vlsr,
    a=a, 
    b=b, 
    pa=pa, 
    dvlsr=dvlsr, 
    kda=kda,
    factor_a=factor_a, 
    factor_b=factor_b, 
    factor_dvlsr=factor_dvlsr, 
    offset_pa=offset_pa,
    unit_a=unit_a, 
    unit_b=unit_b, 
    unit_dvlsr=unit_dvlsr, 
    add_cols=['Dist_lit', 'KDA', 'f_avg', 'ID', 'in_grs'])

table.write(
    os.path.join(dirpath_kda_info, kda_table_name + '.dat'),
    format='ascii', 
    overwrite=True)

Finally, we need to produce a .ini-file giving specifications for the KDA info table. See Appendix A in Riener et al. (2020b) for more information about the parameters.

In [ ]:
reference = 'RD+09'
weight_cat = 0.5
threshold_spatial = 0.9
threshold_spectral = 0.125
size = 'fwhm'
linewidth = 'fwhm'

text_ini = kda_info_table_ini(
    reference, 
    weight_cat=weight_cat, 
    threshold_spatial=threshold_spatial, 
    threshold_spectral=threshold_spectral, 
    size=size, 
    linewidth=linewidth)
with open(os.path.join(dirpath_kda_info, kda_table_name + '.ini'), 'w') as wfile:
    wfile.write(text_ini)